In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("TensorFlow version: {}".format(tf.__version__))
print("If tensorflow version is 1.x, use tf.enable_eager_execution()")

TensorFlow version: 2.2.0
If tensorflow version is 1.x, use tf.enable_eager_execution()


Data Source: <a href="http://ai.stanford.edu/~amaas/data/sentiment/"> IMDB Dataset </a>

In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [3]:
train_data, test_data = imdb['train'], imdb['test']

The imdb reviews loaded from tensorflow datasets are not in the usable format. To convert them into rather simple and more easily readable formats, we have initialised empty lists.

In [4]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

#### Let's now move the contents from train_data and test_data into the lists

In [5]:
for sentences, labels in train_data:
    training_sentences.append(str(sentences.numpy()))
    training_labels.append(labels.numpy())

In [6]:
for sentences, labels in test_data:
    testing_sentences.append(str(sentences.numpy()))
    testing_labels.append(labels.numpy())

#### Create Numpy arrays for the labels

In [7]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

#### Tokenize the sentences

In [8]:
## Let's use this cell to introduce the hyperparameters. This way it would be easier to change them.
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='post'
oov_tok="<OOV>"

In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [10]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, truncating=trunc_type)

Initially, in the word index dictionary, the words are keys and the word tokens are the values. We are going to reverse that dictionary

In [14]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[1])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of i

#### Let's build the neural network

In [15]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(6, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


#### Fit the model

In [16]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 2s 3ms/step - loss: 0.4827 - accuracy: 0.7506 - val_loss: 0.3808 - val_accuracy: 0.8288
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.2385 - accuracy: 0.9086 - val_loss: 0.4096 - val_accuracy: 0.8219
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0970 - accuracy: 0.9749 - val_loss: 0.4995 - val_accuracy: 0.8105
Epoch 4/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0259 - accuracy: 0.9964 - val_loss: 0.5918 - val_accuracy: 0.8084
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0057 - accuracy: 0.9996 - val_loss: 0.6793 - val_accuracy: 0.8092
Epoch 6/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.7350 - val_accuracy: 0.8107
Epoch 7/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.7912 - val_accuracy: 0.8098
Epoch 

In [18]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


#### Get the word embeddings

In [26]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [27]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('Data/vecs.tsv')
  files.download('Data/meta.tsv')